In [1]:
import pymongo
import pprint
import bson.objectid
from datetime import datetime

In [6]:
# Replace XXXX with your connection URI from the Atlas UI
client = pymongo.MongoClient(XXX)
comments = client.sample_mflix.comments

In [3]:
# fake comment
comment = {
    "name": "some users's name",
    "email": "someuser@email.com",
    "movie_id": bson.objectid.ObjectId(),
    "text": "some nice comment on our movie",
    "date": datetime.utcnow()
}

In [4]:
bypass_validation = False

In [7]:
# inserting the fake comment among the comments collection
insert_comment = comments.insert_one(comment, bypass_validation)

In [8]:
# checking if the comment was inserted successfully
pprint.pprint(insert_comment.acknowledged)

True


In [10]:
# printing the id of the inserted comment
pprint.pprint(insert_comment.inserted_id)

ObjectId('63163cc0219df906833b15e6')


In [12]:
# now , we will be providing a custom id to the document
comment = {
    "_id": "some_id_field",
    "name": "some users's name",
    "email": "someuser@email.com",
    "movie_id": bson.objectid.ObjectId(),
    "text": "Hi, it's me again!",
    "date": datetime.utcnow()
}

insert_comment = comments.insert_one(comment, bypass_validation)
pprint.pprint(insert_comment.acknowledged)
pprint.pprint(insert_comment.inserted_id)


True
'some_id_field'


In [13]:
# to make sure the document was inserted successfully even though it has 
# a custom id, we will try to insert it once more

comment = {
    "_id": "some_id_field"
}
insert_result = comments.insert_one(comment, bypass_validation)

# this will raise DuplicateKeyError which means that a similar document
# has the same id, which means that it is possible to have a custom id

DuplicateKeyError: E11000 duplicate key error collection: sample_mflix.comments index: _id_ dup key: { _id: "some_id_field" }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': 'some_id_field'}, 'errmsg': 'E11000 duplicate key error collection: sample_mflix.comments index: _id_ dup key: { _id: "some_id_field" }'}

## Remarks about updating a document ##

In [15]:
# in the video, the user uploaded a new comment to a certain movie
# at first, we prepare a comment document with all its objects
# then , in the update_doc , we will provide it with update fields
# "$inc" will increment the value of num_mflix_comments
# this helps us to not track this value each and every time
# "$inc" will automatically increase the provided value
# the second field is "$push" and this helps us to not replace all the array of comments
# with that specific comment, but rather, push the comment to that array
# and this saves us the trouble of providing all the array once again
# it will push that new value directly